เพิ่มdb

In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="LLM",
    user="postgres",
    password="Guyza5521"
)

cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        title TEXT,
        duration_days INTEGER,
        cities TEXT[],
        months TEXT[],
        content TEXT,
        embedding vector(1024)
    )
""")
conn.commit()
cur.close()
conn.close()

add data

mark 1

In [2]:
from sentence_transformers import SentenceTransformer


embedder = SentenceTransformer("BAAI/bge-m3")
conn = psycopg2.connect(
    host="localhost",
    database="LLM",
    user="postgres",
    password="Guyza5521"
)

cur = conn.cursor()
def add_document(text):
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()

import pandas as pd

# read by default 1st sheet of an excel file
dataframe1 = pd.read_excel('rag_data_eng.xlsx')

for index, row in dataframe1.iterrows():
    # print(index, row.to_dict())
    add_document(str(row.to_dict()))
# add_document('ชมพระนอนทองสัมฤทธิ์ขนาดใหญ่ "วัดนันโซอิน" ที่สุดของความขลัง "ศาลเจ้าเมคาริ" เช็คอินมุมถ่ายรูปสุดชิคที่ใหม่!! "เสาโทริอิกับวิวสะพาน Kanmon Bridge" พิเศษ!! ชมปราสาทโคคุระ และเมนูพิเศษ!! บุฟเฟ่ต์ปิ้งย่าง สไตล์ญี่ปุ่น พักโรงแรมออนเซ็น 1 คืน')
cur.close()
conn.close()

c:\Users\WINDOWS 11\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

mark 2

In [1]:
from sentence_transformers import SentenceTransformer
import ast
import psycopg2
embedder = SentenceTransformer("BAAI/bge-m3")
conn = psycopg2.connect(
    host="localhost",
    database="LLM",
    user="postgres",
    password="password"
)

cur = conn.cursor()
def add_document(row_dict):
    # สร้างข้อความฝัง (embedding) จาก content หรือรวมข้อมูล
    text_for_embedding = f'{{"Title": "{row_dict["Title"]}", "Itinerary": {{{row_dict["Itinerary"].replace("\n", "").strip()}}}}}'

    embedding = embedder.encode(text_for_embedding).tolist()

    # INSERT ให้ตรง type ของ column
    cur.execute("""
        INSERT INTO documents (title, duration_days, cities, months, content, embedding)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row_dict['Title'],
        row_dict['Duration Days'],
        ast.literal_eval(row_dict['Cities']),
        ast.literal_eval(row_dict['Best Months']),
        text_for_embedding,
        embedding
    ))
    conn.commit()

import pandas as pd

# read by default 1st sheet of an excel file
dataframe1 = pd.read_excel('rag_data_eng_upgrade_2.xlsx')

for index, row in dataframe1.iterrows():
    # print(index, row.to_dict())
    add_document(row.to_dict())
# add_document('ชมพระนอนทองสัมฤทธิ์ขนาดใหญ่ "วัดนันโซอิน" ที่สุดของความขลัง "ศาลเจ้าเมคาริ" เช็คอินมุมถ่ายรูปสุดชิคที่ใหม่!! "เสาโทริอิกับวิวสะพาน Kanmon Bridge" พิเศษ!! ชมปราสาทโคคุระ และเมนูพิเศษ!! บุฟเฟ่ต์ปิ้งย่าง สไตล์ญี่ปุ่น พักโรงแรมออนเซ็น 1 คืน')
cur.close()
conn.close()

d:\study\finalproject\final\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


test query

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="LLM",
    user="postgres",
    password="password"
)

cur = conn.cursor()
def query_documents(query_text, k=5):
    query_embedding = embedder.encode(query_text).tolist()
    query_embedding_str = "[" + ",".join(map(str, query_embedding)) + "]"
    query = """
        SELECT content, embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s
    """
    cur.execute(query, (query_embedding_str, k))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

query_documents("4 วัน 3 คืน ผามันซาโมะ ฟูจิ")   

[('4 วัน 3 คืน นำท่านท่องเที่ยวชมทะเลญี่ปุ่น พาท่านไปยังผามันซาโมะ ผาหินที่สูงกว่า 30 เมตรมีลักษณะคล้ายงวงช้างหันหน้าสู่ทะเลจีน พิพิธภัณฑ์สัตว์น้ำชุราอูมิ พิพิธภัณฑ์สัตว์น้ำที่ดีที่ติดอันดับ 1 ใน 3 ของประเทศญี่ปุ่น ชม ปราสาทชูริ (ด้านนอก) ปราสาทตามแบบสถาปัตยกรรมของชาวริวกิว ชม ถ้ำธารมรกตเกียวคุเซ็นโด ถ้ำหินงอก และหินย้อยสวยงาม ยาวกว่า 700 เมตร ศูนย์การค้าอุมิคะจิ เทอเรส ศูนย์การค้าแห่งใหม่ที่สร้างอยู่บนเนินเขาลาดเอียงบนเกาะเซะนะกะจิมะ ศาลเจ้านะมิโนะอุเอะ ชาวญี่ปุ่นมักจะมาไหว้ขอพรเกี่ยวกับธุรกิจการงานให้เจริญรุ่งเรือง ร่ำรวย อเมริกัน วิลเลจ สร้างเลียนแบบแหล่งช้อปปิ้งย่านซานดิเอโก้ในประเทศสหรัฐอเมริกา ',
  0.35361194610595703)]

test prompt

In [5]:
import ollama
conn = psycopg2.connect(
    host="localhost",
    database="LLM",
    user="postgres",
    password="password"
)

cur = conn.cursor()
def query_documents(query_text, k=5):
    query_embedding = embedder.encode(query_text).tolist()
    query_embedding_str = "[" + ",".join(map(str, query_embedding)) + "]"
    query = """
        SELECT content, embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s
    """
    cur.execute(query, (query_embedding_str, 1))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results
def generate_response(text):
    retrieved_docs = query_documents(text)
    context = "\n".join([i[0] for i in retrieved_docs])
    prompt = f"Make an time schedule for a trip based on the following context : {context}"
    response = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content" : "You are an assistant that helps to make a time schedule for a trip."},
        {"role": "user", "content" : prompt},
    ])
    return response["message"]["content"]

generate_response("5 วัน 4 คืน ฟูจิ")

'ยินดีต้อนรับสู่การวางแผนการเดินทางของคุณ!\n\n**วันที่ 1: ริมทะเลสาบคาวากุจิโกะ และหมู่บ้านน้ำใส**\n\n* 08:00 - นอนหลับ\n* 09:00 - อาหารเช้า\n* 10:00 - เดินเล่นหมู่บ้านน้ำใส\n* 12:00 - อาหารกลางวัน\n* 13:30 - ริมทะเลสาบคาวากุจิโกะ\n* 16:00 - การรักไก และการผ่อนคลาย\n* 19:00 -อาหารเย็น\n\n**วันที่ 2: เมืองโตเกียว**\n\n* 08:00 - นอนหลับ\n* 09:00 - อาหารเช้า\n* 10:00 - โอดาวาระ\n* 12:30 - ปราสาทและศาลเจ้าฟูจิซัง\n* 14:00 - ฮงกุ เซ็นเก็น และไทฉะ\n* 17:00 - วัดอาซากุสะ และการไหว้\n* 20:00 - อาหารเย็น\n\n**วันที่ 3: เมืองชิสุโอกะ**\n\n* 08:00 - นอนหลับ\n* 09:00 - อาหารเช้า\n* 10:00 - ไร่ชาเขียวพร้อมวิวเม้าท์ฟูจิ\n* 12:30 - ตลาดปลาสึกิจิ และอาหารทะเลสดๆ\n* 16:00 - เดินเล่นและผ่อนคลาย\n* 19:00 - อาหารเย็น\n\n**วันที่ 4: เมืองฟูจิแช่ออนเซ็น**\n\n* 08:00 - นอนหลับ\n* 09:00 - อาหารเช้า\n* 10:00 - ช้อปปิ้งย่านดังชินจูกุ และ La Cittadella / ไดเวอร์ซิตี้\n* 13:30 - บุฟเฟ่ต์ขาปู และเซ็ตชาบู\n* 16:00 - สไตล์ญี่ปุ่นและบรรยากาศอิตาเลี่ยน\n* 19:00 - อาหารเย็น\n\n**วันที่ 5: วัดคาวาซากิ ไดชิ และการผ่อนคลา